In [ ]:
import pandas as pd
import csv
csv.QUOTE_NONE
# data = pd.read_csv('dev_tweets.txt', header = None)
df = pd.read_csv('train_tweets.txt', sep="\n",encoding='latin-1', header=None, quoting=csv.QUOTE_NONE)
dfPred = pd.read_csv('test_tweets.txt', sep="\n",encoding='latin-1', header=None, quoting=csv.QUOTE_NONE)
# data = pd.read_csv('dev_tweets.txt', sep=",", header=None)
df.columns = ["row"]
dfPred.columns = ["row"]
# print (df)

In [ ]:
tmp_dfPred = dfPred.pop('row').str.split(',')
dfPred['tweet_id'], dfPred['user_tweet'], dfPred['loc'] = tmp_dfPred.str[0], tmp_dfPred.str[1:-1].str.join(sep=','), tmp_dfPred.str[-1]

# print(dfPred)

In [ ]:
# v = data.pop('row').str.split(',', expand=True)
# v.columns = ['tweet_id', 'user_tweet', 'loc']
# data[['tweet_id','user_tweet']] = data.pop('row').str.split(',', n=1, expand=True)
# data[['user_tweet','loc']] = data['user_tweet'].str.rsplit(',', n=1, expand=True)
# print (data)

tmp_df = df.pop('row').str.split(',')
df['tweet_id'], df['user_tweet'], df['loc'] = tmp_df.str[0], tmp_df.str[1:-1].str.join(sep=','), tmp_df.str[-1]

# print(df)

In [ ]:
df['user_id'], df['tweet_text'] = df['user_tweet'].str.split(',', 1).str

In [ ]:
dfPred['user_id'], dfPred['tweet_text'] = dfPred['user_tweet'].str.split(',', 1).str

In [ ]:
df = df[['tweet_id','user_id','tweet_text','loc','user_tweet']]

In [ ]:
dfPred = dfPred[['tweet_id','user_id','tweet_text','loc','user_tweet']]

In [ ]:
df = df.drop('user_tweet', 1)
df

In [ ]:
dfPred = dfPred.drop('user_tweet', 1)
dfPred

In [ ]:
df.to_csv('tweet_csv.csv', encoding='utf-8', index=False)

In [ ]:
dfPred.to_csv('tweet_csv.csv', encoding='utf-8', index=False)

In [ ]:
from collections import Counter
import nltk
import pandas as pd
import re as regex
import numpy as np
import plotly
from plotly import graph_objs
import plotly.offline as pyo
import plotly.graph_objs as go
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from time import time


In [ ]:

ny = len(df[df["loc"] == "NewYork"])
ga = len(df[df["loc"] == "Georgia"])
ca = len(df[df["loc"] == "California"])

In [ ]:
print(ny)
print(ga)
print(ca)

In [ ]:
dist = [
    graph_objs.Bar(
        x=["NewYork","Georgia","California"],
        y=[ny, ga, ca],
)]
plotly.offline.iplot({"data":dist, "layout":graph_objs.Layout(title="Sentiment type distribution in training set")})

In [ ]:
df["tweet_text"]

In [ ]:
dfPred["tweet_text"]

In [ ]:
def count_by_lambda(expression, word_array):
    return len(list(filter(expression, word_array)))


def count_occurences(character, word_array):
    counter = 0
    for (j, word) in enumerate(word_array):
        for char in word:
            if char == character:
                counter += 1

    return counter


def count_by_regex(regex, plain_text):
    return len(regex.findall(plain_text))


def addNewColumns():
    add_column(df, 'splitted_text', map(lambda txt: txt.split(' '),
               df['tweet_text']))

            # number of uppercase words

    uppercase = list(map(lambda txt: count_by_lambda(lambda word: word \
                     == word.upper(), txt), df['splitted_text']))
    add_column(df, 'number_of_uppercase', uppercase)

            # number of !

    exclamations = list(map(lambda txt: count_occurences('!', txt),
                        df['splitted_text']))
    add_column(df, 'number_of_exclamation', exclamations)

            # number of ?

    questions = list(map(lambda txt: count_occurences('?', txt),
                     df['splitted_text']))
    add_column(df, 'number_of_question', questions)

            # number of ...

    ellipsis = list(map(lambda txt: \
                    count_by_regex(regex.compile(r"\.\s?\.\s?\."),
                    txt), df['tweet_text']))
    add_column(df, 'number_of_ellipsis', ellipsis)

            # number of hashtags

    hashtags = list(map(lambda txt: count_occurences('#', txt),
                    df['splitted_text']))
    add_column(df, 'number_of_hashtags', hashtags)

            # number of mentions

    mentions = list(map(lambda txt: count_occurences('@', txt),
                    df['splitted_text']))
    add_column(df, 'number_of_mentions', mentions)

            # number of quotes

    quotes = list(map(lambda plain_text: int(count_occurences("'",
                  [plain_text.strip("'").strip('"')]) / 2
                  + count_occurences('"', [plain_text.strip("'"
                  ).strip('"')]) / 2), df['tweet_text']))
    add_column(df, 'number_of_quotes', quotes)

            # number of urls

    urls = list(map(lambda txt: \
                count_by_regex(regex.compile(r"http.?://[^\s]+[\s]?"),
                txt), df['tweet_text']))
    add_column(df, 'number_of_urls', urls)


def add_column(df, column_name, column_content):
    df.loc[:, column_name] = pd.Series(column_content, index=df.index)


addNewColumns()

In [ ]:
def count_by_lambda(expression, word_array):
    return len(list(filter(expression, word_array)))


def count_occurences(character, word_array):
    counter = 0
    for (j, word) in enumerate(word_array):
        for char in word:
            if char == character:
                counter += 1

    return counter


def count_by_regex(regex, plain_text):
    return len(regex.findall(plain_text))


def addNewColumns():
    add_column(dfPred, 'splitted_text', map(lambda txt: txt.split(' '),
               dfPred['tweet_text']))

            # number of uppercase words

    uppercase = list(map(lambda txt: count_by_lambda(lambda word: word \
                     == word.upper(), txt), dfPred['splitted_text']))
    add_column(dfPred, 'number_of_uppercase', uppercase)

            # number of !

    exclamations = list(map(lambda txt: count_occurences('!', txt),
                        dfPred['splitted_text']))
    add_column(dfPred, 'number_of_exclamation', exclamations)

            # number of ?

    questions = list(map(lambda txt: count_occurences('?', txt),
                     dfPred['splitted_text']))
    add_column(dfPred, 'number_of_question', questions)

            # number of ...

    ellipsis = list(map(lambda txt: \
                    count_by_regex(regex.compile(r"\.\s?\.\s?\."),
                    txt), dfPred['tweet_text']))
    add_column(dfPred, 'number_of_ellipsis', ellipsis)

            # number of hashtags

    hashtags = list(map(lambda txt: count_occurences('#', txt),
                    dfPred['splitted_text']))
    add_column(dfPred, 'number_of_hashtags', hashtags)

            # number of mentions

    mentions = list(map(lambda txt: count_occurences('@', txt),
                    dfPred['splitted_text']))
    add_column(dfPred, 'number_of_mentions', mentions)

            # number of quotes

    quotes = list(map(lambda plain_text: int(count_occurences("'",
                  [plain_text.strip("'").strip('"')]) / 2
                  + count_occurences('"', [plain_text.strip("'"
                  ).strip('"')]) / 2), dfPred['tweet_text']))
    add_column(dfPred, 'number_of_quotes', quotes)

            # number of urls

    urls = list(map(lambda txt: \
                count_by_regex(regex.compile(r"http.?://[^\s]+[\s]?"),
                txt), dfPred['tweet_text']))
    add_column(dfPred, 'number_of_urls', urls)


def add_column(dfPred, column_name, column_content):
    dfPred.loc[:, column_name] = pd.Series(column_content, index=dfPred.index)


addNewColumns()

In [ ]:
# dfTrain.to_csv('trainExtraColumns.csv', index=False)

In [ ]:
# dfTest.to_csv('testExtraColumns.csv', index=False)

In [ ]:
def remove_by_regex(reg,tweets):
    tweets = regex.sub(reg,'',tweets)
    return tweets
def remove_usernames(tweets):
    return remove_by_regex('@[^\s]+[\s]?', tweets)
def remove_urls(tweets):
    return remove_by_regex('http.?://[^\s]+[\s]?', tweets)
def remove_numbers(tweets):
    return remove_by_regex('\s?[0-9]+\.?[0-9]*', tweets)
def remove_special_chars(tweets):
    return remove_by_regex('\\|\?|\.|\!|\/|\;|\:|$|#|%|&|,|\'|]|\|-|\"|_|\[|\||{|', tweets)
def remove_otherspecial_chars(tweets):
    return remove_by_regex('[-?_)(*@=~]+', tweets)
def callAll(tweets):
    tweets = remove_usernames(tweets)
    tweets = remove_urls(tweets)
    tweets = remove_special_chars(tweets)
    tweets = remove_otherspecial_chars(tweets)
    tweets = remove_numbers(tweets)
    return tweets
for i, row_value in df['tweet_text'].iteritems():
    tweets = df['tweet_text'][i]
    df['tweet_text'][i] = callAll(tweets)



In [ ]:
def remove_by_regex(reg,tweets):
    tweets = regex.sub(reg,'',tweets)
    return tweets
def remove_usernames(tweets):
    return remove_by_regex('@[^\s]+[\s]?', tweets)
def remove_urls(tweets):
    return remove_by_regex('http.?://[^\s]+[\s]?', tweets)
def remove_numbers(tweets):
    return remove_by_regex('\s?[0-9]+\.?[0-9]*', tweets)
def remove_special_chars(tweets):
    return remove_by_regex('\\|\?|\.|\!|\/|\;|\:|$|#|%|&|,|\'|]|\|-|\"|_|\[|\||{|', tweets)
def remove_otherspecial_chars(tweets):
    return remove_by_regex('[-?_)(*@=~]+', tweets)
def callAllPred(tweets):
    tweets = remove_usernames(tweets)
    tweets = remove_urls(tweets)
    tweets = remove_special_chars(tweets)
    tweets = remove_otherspecial_chars(tweets)
    tweets = remove_numbers(tweets)
    return tweets
for i, row_value in dfPred['tweet_text'].iteritems():
    tweets = dfPred['tweet_text'][i]
    dfPred['tweet_text'][i] = callAllPred(tweets)


In [ ]:
# def remove_by_regex(tweets):
#     tweets = regex.sub('@[^\s]+[\s]?','',tweets)
#     return tweets
# def remove_usernames(tweets):
#     return remove_by_regex(tweets)
# for i, row_value in df['tweet_text'].iteritems():
#     tweets = df['tweet_text'][i]
#     df['tweet_text'][i] = regex.sub('@[^\s]+[\s]?','',tweets)



# def remove_by_regex(reg,tweets):
#     tweets = regex.sub(reg,'',tweets)
#     return tweets
# def remove_usernames(tweets):
#     return remove_by_regex('@[^\s]+[\s]?', tweets)
# for i, row_value in df['tweet_text'].iteritems():
#     tweets = df['tweet_text'][i]
#     df['tweet_text'][i] = remove_usernames(tweets)

In [ ]:
for i, row_value in df['tweet_text'].iteritems():
        print(df['tweet_text'][i])

In [ ]:
for i, row_value in dfPred['tweet_text'].iteritems():
        print(dfPred['tweet_text'][i])

In [ ]:
dfPred.head(5)

In [ ]:
df.head(5)

In [ ]:
def tokenize(df, tokenizer=nltk.word_tokenize):
        def tokenize_row(row):
            row["tweet_text"] = tokenizer(row["tweet_text"])
            row["tokenized_text"] = [] + row["tweet_text"]
            return row

        df = df.apply(tokenize_row, axis=1)
tokenize(df)

# for i, row_value in df['tweet_text'].iteritems():
#     tweets = df['tweet_text'][i]
#     df['tweet_text'][i] = callAll(tweets)

In [ ]:
def tokenize(dfPred, tokenizer=nltk.word_tokenize):
        def tokenize_row(row):
            row["tweet_text"] = tokenizer(row["tweet_text"])
            row["tokenized_text"] = [] + row["tweet_text"]
            return row

        dfPred = dfPred.apply(tokenize_row, axis=1)
tokenize(dfPred)

In [ ]:
def stem(df, stemmer=nltk.PorterStemmer()):
        def stem_and_join(row):
            row["tweet_text"] = list(map(lambda str: stemmer.stem(str.lower()), row["tweet_text"]))
            return row

        df = df.apply(stem_and_join, axis=1)
stem(df)

In [ ]:
def stem(dfPred, stemmer=nltk.PorterStemmer()):
        def stem_and_join(row):
            row["tweet_text"] = list(map(lambda str: stemmer.stem(str.lower()), row["tweet_text"]))
            return row

        dfPred = dfPred.apply(stem_and_join, axis=1)
stem(dfPred)

In [ ]:
words = Counter()
for idx in df.index:
    words.update(df.loc[idx, "tweet_text"])

words.most_common(5)

In [ ]:
stopwords=nltk.corpus.stopwords.words("english")
for idx, stop_word in enumerate(stopwords):
    del words[stop_word]
words.most_common(5)

In [ ]:
wordlist = []
        
def build_wordlist(df, min_occurrences=10, max_occurences=10000, stopwords=nltk.corpus.stopwords.words("english"),whitelist=None):
    wordlist = []
    whitelist = whitelist if whitelist is None else whitelist
    import os
    if os.path.isfile("data\\wordlistNew.csv"):
        word_df = pd.read_csv("data\\wordlistNew.csv")
        word_df = word_df[word_df["occurrences"] > min_occurrences]
        wordlist = list(word_df.loc[:, "word"])
        return wordlist

    words = Counter()
    for idx in df.index:
        words.update(df.loc[idx,"tweet_text"])

    for idx, stop_word in enumerate(stopwords):
        del words[stop_word]

    word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

    word_df.to_csv("data\\wordlistNew.csv", index_label="idx")
    wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
    return wordlist

In [ ]:
wordlist = build_wordlist(df)

In [ ]:
wordlist

In [ ]:
words = pd.read_csv("data\\wordlistNew.csv")
pyo.init_notebook_mode()

x_words = list(words.loc[0:10,"word"])
x_words.reverse()
y_occ = list(words.loc[0:10,"occurrences"])
y_occ.reverse()

dist = [
    go.Bar(
        x=y_occ,
        y=x_words,
        orientation="h"
)]
pyo.iplot({"data":dist, "layout":graph_objs.Layout(title="Top words in built wordlist")})

In [ ]:
    def build_data_model(df,is_testing, wordlist):
        label_column = []
        if not is_testing:
            label_column = ["label"]

        columns = label_column + list(
            map(lambda w: w + "_bow",wordlist))
        labels = []
        rows = []
        for idx in df.index:
            current_row = []

            if not is_testing:
                # add label
                current_label = df.loc[idx, "loc"]
                labels.append(current_label)
                current_row.append(current_label)

            # add bag-of-words
            tokens = set(df.loc[idx, "tweet_text"])
            for _, word in enumerate(wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        data_model = pd.DataFrame(rows, columns=columns)
        data_labels = pd.Series(labels)
        return data_model, data_labels

In [ ]:
# data = TwitterData_BagOfWords(data)
bow, labels = build_data_model(df, False, wordlist)
bowP, labelsP = build_data_model(dfPred, True, wordlist)
bowP.head(5)
bow.head(5)

In [ ]:
dfEXtra = pd.read_csv('trainExtraColumns.csv')

In [ ]:
dfEXtra

In [ ]:
dfPredExtra = pd.read_csv('testExtraColumns.csv')

In [ ]:
dfPredExtra

In [ ]:
nyDf = pd.read_csv('NYWords', sep="\n",encoding='latin-1', header=None, quoting=csv.QUOTE_NONE)

In [ ]:
caDf = pd.read_csv('caliWords', sep="\n",encoding='latin-1', header=None, quoting=csv.QUOTE_NONE)

In [ ]:
gaDf = pd.read_csv('geoWords', sep="\n",encoding='latin-1', header=None, quoting=csv.QUOTE_NONE)

In [ ]:
nyDf = nyDf[0].values.tolist()

In [ ]:
caDf = caDf[0].values.tolist()

In [ ]:
gaDf = gaDf[0].values.tolist()

In [ ]:
allWords = nyDf + caDf + gaDf
allWords

In [ ]:
for word in allWords:
    dfPred["is"+word] = np.where(dfPred['tweet_text'].str.contains(word), 1, 0)






# def count_by_regex(regex, plain_text):
#     return len(regex.findall(plain_text))


# def addNewColumns():
    


In [ ]:
def count_by_lambda(expression, word_array):
    return len(list(filter(expression, word_array)))
def count_occurences(character, word_array):
    counter = 0
    for (j, word) in enumerate(word_array):
        for char in word:
            if char == character:
                counter += 1

    return counter

def add_column(dfPred, column_name, column_content):
    dfPred.loc[:, column_name] = pd.Series(column_content, index=dfPred.index)
# add_column(df, 'splitted_text', map(lambda txt: txt.split(' '),
#                df['tweet_text']))

for word in allWords:
    count = list(map(lambda txt: count_occurences(word, txt),
                     dfPred['tweet_text']))

    add_column(dfPred, 'is'+word, count)
    


In [ ]:
for i, row_value in dfPred['tweet_text'].iteritems():
    tweets = dfPred['tweet_text'][i]
#     print(tweets)
    for word in allWords:
#         print(word)
#         print[]
        if(any(i.lower() == word.lower() for i in tweets)):
            print("inside if setting 1")
            dfPred['is'+word][i] = 1
        
    
# df['tweet_text'][0]

In [ ]:
for i, row_value in df['tweet_text'].iteritems():
    tweets = df['tweet_text'][i]
#     print(tweets)
    for word in allWords:
#         print(word)
#         print[]
        if(any(i.lower() == word.lower() for i in tweets)):
            print("inside if setting 1")
            df['is'+word][i] = 1
        
    
# df['tweet_text'][0]

In [ ]:
featuresdf = df.loc[:, 'isDeadass':]

In [ ]:
featuresdfPred = dfPred.loc[:, 'isDeadass':]

In [ ]:
# print(labels)
grouped = bow.groupby(["label"]).sum()
# print(grouped)
words_to_visualize = []
sentiments = ["NewYork","Georgia","California"]
#get the most 7 common words for every sentiment
for sentiment in sentiments:
    words = grouped.loc[sentiment,:]
    words.sort_values(inplace=True,ascending=False)
    for w in words.index[:7]:
        if w not in words_to_visualize:
            words_to_visualize.append(w)
            
            
#visualize it
plot_data = []
for sentiment in sentiments:
    plot_data.append(go.Bar(
            x = [w.split("_")[0] for w in words_to_visualize],
            y = [grouped.loc[sentiment,w] for w in words_to_visualize],
            name = sentiment
    ))
    
pyo.iplot({
        "data":plot_data,
        "layout":go.Layout(title="Most common words across sentiments")
    })

In [ ]:
bowFinal = pd.concat([bow.reset_index(drop=True),dfEXtra.reset_index(drop=True), featuresdf.reset_index(drop=True)], axis=1)

In [ ]:
bowFinal = bowFinal.drop('tweet_id', 1)

In [ ]:
bowPredFinal = pd.concat([bowP.reset_index(drop=True),dfPredExtra.reset_index(drop=True), featuresdfPred.reset_index(drop=True)], axis=1)

In [ ]:
bowPredFinal = bowPredFinal.drop('tweet_id', 1)

In [ ]:
bowFinal = bowFinal.drop('number_of_question', 1)

In [ ]:
bowPredFinal = bowPredFinal.drop('number_of_question', 1)

In [ ]:
allWords

In [ ]:
sentiments = ["NewYork","Georgia","California"]
plots_data_ef = []
for what in map(lambda o: "number_of_"+o,["exclamation"]):
# for what in map(lambda o: "is"+o,allWords):
    ef_grouped = bowFinal[bowFinal[what]>=1].groupby(["label"]).count()
    plots_data_ef.append({"data":[graph_objs.Bar(
            x = sentiments,
            y = [ef_grouped.loc[s,:][0] for s in sentiments],
    )], "title":"How feature \""+what+"\" separates the tweets"})
    

for plot_data_ef in plots_data_ef:
    plotly.offline.iplot({
            "data":plot_data_ef["data"],
            "layout":graph_objs.Layout(title=plot_data_ef["title"])
    })

In [ ]:
import random
seed = 5
random.seed(seed)

In [ ]:
def test_classifier(X_train, y_train, X_test, y_test,bowP, classifier):
    log("")
    log("===============================================")
    classifier_name = str(type(classifier).__name__)
    log("Testing " + classifier_name)
    now = time()
    list_of_labels = sorted(list(set(y_train)))
    model = classifier.fit(X_train, y_train)
    log("Learing time {0}s".format(time() - now))
    now = time()
    predictions = model.predict(X_test)
    predictionsForTest = model.predict(bowP)
    print(predictionsForTest)
    log("Predicting time {0}s".format(time() - now))

    precision = precision_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    recall = recall_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    log("=================== Results ===================")
    log("NewYork     Georgia     California")
    log("F1       " + str(f1))
    log("Precision" + str(precision))
    log("Recall   " + str(recall))
    log("Accuracy " + str(accuracy))
    log("===============================================")

    return precision, recall, accuracy, f1, predictionsForTest

def log(x):
    #can be used to write to log file
    print(x)

In [ ]:
# from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
# from xgboost import XGBClassifier as LogisticRegression
# from sklearn.naive_bayes import BernoulliNB
X_train, X_test, y_train, y_test = train_test_split(bow.iloc[:, 1:], bow.iloc[:, 0],train_size=0.65,
                                                   random_state=seed)
print('X_train')
print(X_train)
print('X_test')
print(X_test)
print('y_train')

print(y_train)
list_of_labels = sorted(list(set(y_train)))
print(list_of_labels)
print('y_test')

print(y_test)

precision, recall, accuracy, f1, predictionsForTest = test_classifier(X_train, y_train, X_test, y_test, bowP, KNeighborsClassifier())

In [ ]:
from sklearn.linear_model import SGDClassifier
# from sklearn.naive_bayes import BernoulliNB
X_train, X_test, y_train, y_test = train_test_split(bowFinal.iloc[:, 1:], bowFinal.iloc[:, 0],train_size=0.65,
                                                   random_state=seed)
print('X_train')
print(X_train)
print('X_test')
print(X_test)
print('y_train')

print(y_train)
list_of_labels = sorted(list(set(y_train)))
print(list_of_labels)
print('y_test')

print(y_test)

precision, recall, accuracy, f1, predictionsForTest = test_classifier(X_train, y_train, X_test, y_test, bowPredFinal, SGDClassifier())

In [ ]:
from sklearn.naive_bayes import BernoulliNB
X_train, X_test, y_train, y_test = train_test_split(bowFinal.iloc[:, 1:], bowFinal.iloc[:, 0],train_size=0.65,
                                                   random_state=seed)
print('X_train')
print(X_train)
print('X_test')
print(X_test)
print('y_train')

print(y_train)
list_of_labels = sorted(list(set(y_train)))
print(list_of_labels)
print('y_test')

print(y_test)

precision, recall, accuracy, f1, predictionsForTest = test_classifier(X_train, y_train, X_test, y_test, bowPredFinal, BernoulliNB())

In [ ]:
for item in predictionsForTest:
    print(item)


In [ ]:
np.savetxt(r'outputPredXGBNoHashNoQWords.txt', predictionsForTest, fmt='%s')

In [ ]:
text_file = open("test_tweets_copy.txt", "r")

In [ ]:
lines = [line.rstrip('\n') for line in open('test_tweets_copy.txt', encoding = "ISO-8859-1")]

In [ ]:
lines

In [ ]:
for i in range(len(lines)):
    lines[i] = lines[i][:-1]+predictionsForTest[i]
lines

In [ ]:
linesDf = pd.DataFrame(lines) 

In [ ]:
linesDf.head(5)

In [ ]:
linesDf.columns = ["row"]

In [ ]:
tmp_linesDf = linesDf.pop('row').str.split(',')
linesDf['tweet-id'],linesDf['class'] = tmp_linesDf.str[0],tmp_linesDf.str[-1]

print(linesDf)

In [ ]:
linesDf.to_csv('kaggleOutputSDGNoHashNoQWords.csv', index=False)